# Sequence to Sequence neural models for text translation handout
## [COSC 7336 Advanced Natural Language Processing](https://fagonzalezo.github.io/dl-tau-2017-2/)

In [3]:
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, GRU
from keras.callbacks import ModelCheckpoint

import numpy as np

from keras import backend as K
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

Using TensorFlow backend.


In this handout we build a Neural Machine Translation model based Long-Short Term Memory (LSTM) for sequence to sequence prediction using Keras. This handout is based on the code here (https://github.com/fchollet/keras/blob/master/examples/lstm_seq2seq.py), which is discussed in this [blog](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html).

The model uses two LSTMs, one for encoding a sentence in one language and the other one to generate (decode) a corresponding sentence in the target language. The model is depicted in the following figure:

![encoder-decoder](https://camo.githubusercontent.com/80fdbf3e4aa1ad2c9c0d8bef7c392cbe862ee650/68747470733a2f2f65736369656e636567726f75702e66696c65732e776f726470726573732e636f6d2f323031362f30332f736571327365712e6a70673f773d363235)

However, here we will implement character-based model instead of a word-based model. So it looks more like this model:

![encoder-decoder](https://www.tensorflow.org/images/basic_seq2seq.png)

We will use a paired corpus of English-Spanish sentences available [here](http://www.manythings.org/anki/spa-eng.zip). The following code reads the dataset 

In [75]:
data_path = 'spa-eng/spa.txt' # Path to the data txt file on disk.
num_samples = 100000  # Number of samples to train on.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data_path).read().split('\n')
#for line in lines[: min(num_samples, len(lines) - 1)]:
for line in lines[:num_samples]:
    input_text, target_text = line.split('\t')
    if len(target_text) > 100 or len(input_text) > 100:
        break
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

The following are some statistics from the dataset:

In [76]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 100000
Number of unique input tokens: 86
Number of unique output tokens: 106
Max sequence length for inputs: 46
Max sequence length for outputs: 85


These are some examples of the pairs in the dataset:

In [77]:
for i in range(0,10000, 1000):
    print(input_texts[i],target_texts[i])

Go. 	Ve.

I'm faster. 	Yo soy más rápido.

We help Tom. 	Nosotros ayudamos a Tom.

Who is there? 	¿Quién está ahí?

It works well. 	Funciona bien.

Here comes Tom. 	Aquí viene Tom.

Tom can't walk. 	Tom no puede andar.

I know it's hot. 	Sé que está caliente.

This is for you. 	Esto es para ti.

He became famous. 	Él se hizo famoso.



Characters will be encoded using a one-hot-representation. We also build dictionaries to map from characters to indices and back

In [78]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

print("encoder_input_data shape:", encoder_input_data.shape)
print("decoder_input_data shape:", decoder_input_data.shape)

encoder_input_data shape: (100000, 46, 86)
decoder_input_data shape: (100000, 85, 106)


As depicted in the previous figures, the model uses two LSTM layers. The first one is the encoder. The encoder process the input sentence in the source language, character by character. The output of the layer is not important. Instead, we use it internal state represented by the internal variables $h$ and $c$. In this implementation, we will use the last state, $h_n$ and $c_n$, the state of the LSTM units after processing the $n$ characters of the input. The following code define the encoder:

In [79]:
latent_dim = 256  # Latent dimensionality of the encoding space.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))

encoder = LSTM(latent_dim, return_state=True)

encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

The goal of the decoder is to generate the output sentence in the target language. It uses the internal state of the encoder as initial state for generation. It works as a character-based language model. At each step it generates an output character $y_n$ using as input the previous character $y_{n-1}$. During training, we will use the real targets as input to the decoder instead of the characters generated by the LSTM. This is called *teacher forcing*. So, the decoder also receive the target values as inputs (`decoder_input_data`), but shifted by one time step. The initial state of the decoder is set to the last state of the encoder. The output of the decoder at each time step is a distribution over all the possible output characters (`num_decoder_tokens`), this is performed by a dense layer with a softmax output.

In [80]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

Finally, the model is defined with inputs corresponding to both the encoder and decoder inputs and the output corresponding to the output of the decoder. 

In [81]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_9 (InputLayer)             (None, None, 86)      0                                            
____________________________________________________________________________________________________
input_10 (InputLayer)            (None, None, 106)     0                                            
____________________________________________________________________________________________________
lstm_5 (LSTM)                    [(None, 256), (None,  351232      input_9[0][0]                    
____________________________________________________________________________________________________
lstm_6 (LSTM)                    [(None, None, 256), ( 371712      input_10[0][0]                   
                                                                   lstm_5[0][1]            

Now, we are ready for training the model. Since the output of the decoder is softmax we use a categorical cross entropy loss. The best model so far is stored using a `ModelCheckpoint` callback.

In [27]:
batch_size = 64  # Batch size for training.
epochs = 30  # Number of epochs to train for.

# Run training
callback = ModelCheckpoint('s2s_weights.{epoch:02d}-{val_loss:.2f}.hdf5', 
                           save_best_only=True, save_weights_only=True)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.05,
          callbacks=[callback])


Train on 95000 samples, validate on 5000 samples
Epoch 1/30
95000/95000 [==============================] - 478s 5ms/step - loss: 0.4182 - val_loss: 0.6882
Epoch 2/30
95000/95000 [==============================] - 479s 5ms/step - loss: 0.3813 - val_loss: 0.6485
Epoch 3/30
95000/95000 [==============================] - 478s 5ms/step - loss: 0.3582 - val_loss: 0.6237
Epoch 4/30
95000/95000 [==============================] - 479s 5ms/step - loss: 0.3418 - val_loss: 0.6063
Epoch 5/30
95000/95000 [==============================] - 479s 5ms/step - loss: 0.3295 - val_loss: 0.5987
Epoch 6/30
95000/95000 [==============================] - 479s 5ms/step - loss: 0.3197 - val_loss: 0.5859
Epoch 7/30
95000/95000 [==============================] - 479s 5ms/step - loss: 0.3117 - val_loss: 0.5770
Epoch 8/30
95000/95000 [==============================] - 478s 5ms/step - loss: 0.3048 - val_loss: 0.5737
Epoch 9/30
95000/95000 [==============================] - 479s 5ms/step - loss: 0.2989 - val_loss: 0.57

In [82]:
# This cell load a pretrained model that was trained with 100000 samples
# The values of the following variables have to be set before defining the model:
# Number of unique input tokens: 86
# Number of unique output tokens: 106
# Max sequence length for inputs: 46
# Max sequence length for outputs: 85

model.load_weights('s2s_weights.21-0.55.hdf5')

We define a different model for prediction taking into accournt that during prediction we don't receive inputs for the decoder. Instead we use the output of the decoder from the previous time step. Also we define two separate models for the decoder and the encoder so that we can call them separately:

In [22]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
encoder_model.summary()
decoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, 86)          0         
_________________________________________________________________
lstm_3 (LSTM)                [(None, 256), (None, 256) 351232    
Total params: 351,232
Trainable params: 351,232
Non-trainable params: 0
_________________________________________________________________
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, None, 106)     0                                            
____________________________________________________________________________________________________
input_7 (InputLayer)             (None, 256)           0                                            
___________________________

Now we can use the models for generating translations for input sentences. First we use a a straightforward approach that selects the character with maximum probability at each step of the decoder execution:

In [83]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

We define a function that takes as input a text and encode it using the one-hot-representation:

In [84]:
def one_hot_encoder_input(input_text):
    result = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')    
    for t, char in enumerate(input_text):
        result[0, t, input_token_index[char]] = 1.
    return result

Now we are ready for testing our model:

In [86]:
decode_sequence(one_hot_encoder_input("I didn't know you."))

'No sabía que estoy cansado.\n'

The problem with this straightforward generation strategy is that it is greedy, so it does not necessarly generate the sequence with the highest probability. Generating the maximum-probability sequence is a hard computational problem, however there are strategies that help to deal with the combinatorial explosion. One of the most popular ones is [Beam Search](https://en.wikipedia.org/wiki/Beam_search). The idea is to generate a set of candidate characters with high probability at each step, instead of genearating only the one with the maximum probability, and look ahead a number of stesp.  

The following image illustrates the overall idea: 

![beam search](https://cloud.githubusercontent.com/assets/1135354/12116947/9e9898d2-b3bd-11e5-9f23-b2c8ba03ab52.jpg)

At each stept several candidate characters are considered, the look ahead allows to explore different combinations and choose the one with the highest probability:


The following code implements beam search. The parameters of the function are as follows:

* `states_value`: the value of the internal states of the decoder from the previous step.
* `target_idx`: the index of the las character generated.
* `lahead`: how many steps to look ahead.
* `nbeams`: how many different candidate characters (beams) to consider at each step.
* `decrement`: the number of beams can be decremented at each step by this parameter
* `min_nbeams`: the minimum number of beams, if the number of beams is equal to min_nbeams it is not further decreased.



In [87]:
def beam_search(states_value, target_idx, lahead=3, 
                nbeams = 2, min_nbeams = 1, decrement = 0):
    nbeams = max(nbeams, min_nbeams)
    if reverse_target_char_index[target_idx] == '\n' or lahead < 1:
        return ([], 0, 0)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_idx] = 1.
    output_tokens, h, c = decoder_model.predict(
        [target_seq] + states_value)
    states_value = [h, c]
    # Sample a token
    candidates = [(output_tokens[0, -1, i], i) for i in range(output_tokens.shape[2])]
    candidates.sort()
    best_prob = float('-inf')
    best_idx = 0
    for prob, idx in candidates[-nbeams:]:
        _, _, cand_log_prob = beam_search(states_value, idx, lahead - 1,
                                                           nbeams - decrement, min_nbeams, decrement)
        cand_log_prob += np.log(prob)
        if cand_log_prob > best_prob:
            best_prob = cand_log_prob
            best_idx = idx
    return (states_value, best_idx, best_prob)

def decode_sequence_beam(input_seq, lahead=3, nbeams = 2, min_nbeams = 1, decrement = 0):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    decoded_sentence = ''
    best_idx = target_token_index['\t']
    for i in range(max_decoder_seq_length):
        states_value, best_idx, best_prob = beam_search(states_value, best_idx , 
                                                        nbeams = nbeams, 
                                                        min_nbeams = min_nbeams, 
                                                        decrement = decrement)
        sampled_char = reverse_target_char_index[best_idx]
        if sampled_char == '\n':
            break
        decoded_sentence += sampled_char
    return decoded_sentence


In [92]:
decode_sequence_beam(one_hot_encoder_input("I didn't know you."), lahead=4, nbeams=4)

'No sabía que estoy.'